# Dependencies

In [ ]:
! pip -q install datasets evaluate torch nltk rank_bm25
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

# Dataset

## Import Dataset

In [ ]:
import pandas as pd

df = pd.read_csv("1000rows_with_summary.csv")
df.head()

,Unnamed: 0,query,human answer,llm summaries,llm answer
0,0,what is rba,['Results-Based Accountability is a discipline...,Risk-Based Asset Management (RBA) is an approa...,Risk-Based Authentication (RBA) is an approac...
1,1,was ronald reagan a democrat,['Yes'],Ronald Reagan was a member of the Democratic P...,"No, Ronald Reagan was never a Democrat. He be..."
2,2,how long do you need for sydney and surroundin...,['20-25 minutes'],"The Blue Mountains, Central Coast and national...","It depends on your travel plans, but generall..."
3,3,price to install tile in shower,['$11 to $22 per square foot'],Cost of installing tile in a shower varies wid...,The cost of installing tile in a shower can v...
4,4,why conversion observed in body,['Due to symptoms in the body'],The exact cause of conversion disorder is not ...,The exact cause of conversion disorder is not...


In [ ]:
df.shape

(1000, 5)

## Average Length of summary

In [ ]:
# average length of each summary
def length(textlist):
    import numpy as np
    lengt = []
    for text in textlist:
        if text == None:
            lengt.append(0)
            continue
        lengt.append(len(text.split(" ")))
    return np.mean(lengt)

print(length(df['llm summaries']))

45.486


# Load the Model

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("satyanshu404/bart-large-cnn-prompt_generation")
model_trained = AutoModelForSeq2SeqLM.from_pretrained("satyanshu404/bart-large-cnn-prompt_generation").to(device)

# Supporting Functions

In [ ]:
def add_instruction(text):
  instruction = """Instruction: Based on the given description generate a list of all the prompts that are relavent to the document.
  Query: """
  return f"{instruction} {text}"

In [ ]:
def generate_results(prompts, model, tokenizer):
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
  gen_tokens = model.generate(input_ids).to(device)
  gen_text = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]
  return gen_text

# Inference

## Generate all prompts

In [ ]:
import time

list_queries = []

In [ ]:
start0 = time.time()

for i, summary in enumerate(df['llm summaries']):
  start = time.time()
  print('='*100)
  # extracting each sentence from each summary
  sentences = summary.split('\n')
  queries = []
  # enumerating through each summary
  for sentence in sentences:
    # coverting into prompts
    prompt = add_instruction(sentence)
    # generating queries
    gen_text = generate_results(prompt, model_trained, tokenizer)
    # getting all the queries
    queries.extend(gen_text.split(', '))

  list_queries.append(queries)

  end = time.time()
  print(f'Time for summary {i+1}: {end - start}')

end = time.time()
print(f'Total Time: {end - start0}')

Time for summary 1: 4.0764689445495605
Time for summary 2: 5.0528857707977295
Time for summary 3: 4.687159299850464
Time for summary 4: 4.250669002532959
Time for summary 5: 9.181942701339722
Time for summary 6: 6.131484508514404
Time for summary 7: 5.821340322494507
Time for summary 8: 5.311720609664917
Time for summary 9: 5.874441623687744
Time for summary 10: 3.2433793544769287
Time for summary 11: 5.736355543136597
Time for summary 12: 5.098077297210693
Time for summary 13: 5.514856338500977
Time for summary 14: 5.131215333938599
Time for summary 15: 3.286055326461792
Time for summary 16: 4.231643199920654
Time for summary 17: 7.04622220993042
Time for summary 18: 4.298925876617432
Time for summary 19: 3.5185434818267822
Time for summary 20: 5.200260162353516
Time for summary 21: 3.4891197681427
Time for summary 22: 5.256516695022583
Time for summary 23: 7.569438457489014
Time for summary 24: 4.359593629837036
Time for summary 25: 7.395314455032349
Time for summary 26: 5.6863675117

## Select top 10 using BM25 ranking

In [ ]:
queries = []
scores = []

In [ ]:
from rank_bm25 import BM25Okapi

for corpus, query in zip(list_queries, df['llm summaries']):
  tokenized_corpus = [doc.split(" ") for doc in corpus]
  bm25 = BM25Okapi(tokenized_corpus)

  tokenized_query = query.split(" ")

  doc_scores = bm25.get_scores(tokenized_query)
  tmp = bm25.get_top_n(tokenized_query, corpus, n=10)

  scores.append(doc_scores)
  queries.append(tmp)

In [ ]:
df['llm summaries'][999]

In [ ]:
list_queries[999]

In [ ]:
scores[999]

In [ ]:
queries[999]

## Download as CSV file

In [ ]:
import csv


# Combine the lists into a list of lists
combined_data = list(zip(df['llm summaries'], list_queries, scores, queries))

# Specify the CSV file name
csv_file = 'generated_queries.csv'

# Open the CSV file in write mode
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the header (optional)
    writer.writerow(['llm_summaries', 'All_queries', 'Score', 'top_10_queries'])

    # Write the combined data
    writer.writerows(combined_data)

print(f'CSV file "{csv_file}" has been created successfully.')

CSV file "generated_queries.csv" has been created successfully.


# Sample


In [ ]:
prompt = """
Instruction: Based on the given description generate a list of all the prompts that are relavent to the document.
Query:  Risk-Based Asset Management (RBA) is an approach to applying varying degrees of complexity to authentication procedures."""

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
gen_tokens = model_trained.generate(input_ids, temperature=0.7)
gen_text = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]

print(gen_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Risk-Based Asset Management is an approach to applying varying degrees of complexity to authentication procedures. What is a risk-based asset management approach?, What are some examples?, What is an example of an RBA?, How does it work?, How do I use it in my business?, What does it look like in a bank?, How is it different from a financial institution?, How much does it cost?, How long does it take to build an account?


In [ ]:
df['llm summaries'][4]

'The exact cause of conversion disorder is not yet known.\nIt is caused by an imbalance of brain chemicals.\nThe exact causes of conversion are not known..\nThere is no cure for the disorder.\nBut it can be treated with medication.\nIf you or someone you know is suffering from the disorder, call the National Suicide Prevention Lifeline at 1-800-273-8255.'

In [ ]:
# If whole summary is passed
# Risk-Based Asset Management (RBA) is an approach to applying varying degrees of complexity to authentication procedures.
# It is an attempt to apply varying degrees to each session initiated by a specific user.
# RBA requires transaction-dependent RBA to adapt the required level of authentication.

In [ ]:
# If only one short sentence of summary is passed

# Risk-Based Asset Management is an approach to applying varying degrees of complexity to authentication procedures.
# What is a risk-based asset management approach?,
# What are some examples?,
# What is an example of an RBA?,
# How does it work?,
# How do I use it in my business?,
# What does it look like in a bank?,
# How is it different from a financial institution?,
# How much does it cost?,
# How long does it take to build an account?